<a href="https://colab.research.google.com/github/Sasa-fras/odw/blob/main/Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Lab 2: Gravitational Waves

####Import Stuff

In [1]:
import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
#Tutorial 1.1
! pip install -q 'gwosc==0.7.1'
import gwosc
from gwosc import datasets
from gwosc.datasets import find_datasets
from gwosc.datasets import event_gps
from gwosc.datasets import event_at_gps
from gwosc.datasets import run_segment
from gwosc.datasets import run_at_gps
from gwosc.locate import get_event_urls
from gwosc.datasets import query_events
#Tutorial 1.2 & 1.3
! pip install -q 'gwpy==3.0.12'
import gwpy
from gwpy.timeseries import TimeSeries
from scipy.signal import get_window
#Tutorial 1.4
! pip install -q 'lalsuite==7.25' 'PyCBC==2.4.1'
import matplotlib.pyplot as plt
from pycbc.waveform import get_td_waveform, get_fd_waveform, fd_approximants, td_approximants
#Tutorial 2.1
import numpy
# https://pycbc.org/pycbc/latest/html/noise.html
import pycbc.noise
import pycbc.psd
from pycbc.types import TimeSeries
#Tutorial 2.2
! pip install -q -U 'scipy==1.12.0'
from pycbc.catalog import Merger
from pycbc.filter import resample_to_delta_t, highpass
from pycbc.psd import interpolate, inverse_spectrum_truncation
from pycbc.filter import matched_filter
from pycbc.filter import sigma
from pycbc.frame import read_frame
import os
import urllib
#Tutorial 2.3
# WARNING!! If you are having problems with this code, replace the import with
#from pycbc_chisq import power_chisq
from pycbc.vetoes import power_chisq
from pycbc.events.ranking import newsnr
from pycbc.detector import Detector
#Tutorial 3.1
! pip install -U -q 'bilby==2.4.0' 'matplotlib==3.10.0' 'dynesty==2.1.5' 'corner==2.2.3'
import numpy as np
# The first import of matplotlib can take some time (especially on cloud platforms). This is normal.
import matplotlib
import scipy
import bilby
import corner
#Tutorial 3.2
from bilby.core.prior import Uniform, PowerLaw
from bilby.gw.conversion import convert_to_lal_binary_black_hole_parameters, generate_all_bbh_parameters
#Tutorial 3.3
! pip install -U -q 'astropy==7.0.1'
import h5py
import pandas as pd
import astropy.units as u
from astropy.cosmology import Planck15, z_at_value
#Tutorial 3.4


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 46.9 MB/s eta 0:00:00


In [12]:
#Gather every single event and add it to a database where object oriented programing is used to attribute the correct catalog, run, and observatory to each event

#Create the empty dataframe
DATA = pd.DataFrame({'Catalog':[],'Run':[],'Detector':[],'Event':[],'Time':[],'gps':[]})


#Populate the dataframe
for r in find_datasets(type='run'):
  for c in find_datasets(type='catalog', segment=run_segment(r)):
    for e in find_datasets(type='event', catalog=c, segment=run_segment(r)):
      DATA.loc[len(DATA)] = [c, r, datasets.event_detectors(e), e, gwpy.time.tconvert(event_gps(e)), event_gps(e)]




In [48]:
#sort the datafram by time, and then by event if the times are identical
DATA = DATA.sort_values(by=['Time', 'Event'])
DATA['Catalog'] = DATA['Catalog'].astype(str)
DATA['Run'] = DATA['Run'].astype(str)
DATA['Event'] = DATA['Event'].astype(str)
DATA['gps'] = DATA['gps'].astype(float)

#reindex
DATA = DATA.reset_index(drop=True)

# Set options to display all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000) # Adjust as needed for your console width

print(DATA)

# Reset options to default if needed
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.width')

                 Catalog                Run          Detector               Event                    Time           gps          Object_Name
0     Initial_LIGO_Virgo                 S6          {H1, L1}  blind_injection-v1 2010-09-16 06:42:22.900  9.686546e+08  _blind_injection_v1
1      O1_O2-Preliminary           O1_16KHZ          {H1, L1}         GW150914-v1 2015-09-14 09:50:45.400  1.126259e+09         _GW150914_v1
2      O1_O2-Preliminary                 O1          {H1, L1}         GW150914-v1 2015-09-14 09:50:45.400  1.126259e+09         _GW150914_v1
3      O1_O2-Preliminary           O1_16KHZ          {H1, L1}         GW150914-v2 2015-09-14 09:50:45.400  1.126259e+09         _GW150914_v2
4      O1_O2-Preliminary                 O1          {H1, L1}         GW150914-v2 2015-09-14 09:50:45.400  1.126259e+09         _GW150914_v2
5       GWTC-1-confident           O1_16KHZ          {H1, L1}         GW150914-v3 2015-09-14 09:50:45.400  1.126259e+09         _GW150914_v3
6       GWTC-

In [49]:
#Try using object oriented programming to create a Class called 'event' which can be used to more easily access the other variables associated with each event

class Event:
  def __init__(self, Catalog, Run, Detector, Event, Time, gps):
    self.Catalog = Catalog
    self.Run = Run
    self.Detector = Detector
    self.Event = Event
    self.Time = Time
    self.gps = gps

#if event name has a '-' replace it with a '_' for the event_name
#if event name has a '.' replace it with a '_' for the event_name
#if event name starts with a number, append a '_' in the front of the event_name
#actually, just put a '_' infront of every name to be consistent
for i in range(len(DATA)):
  event_name = DATA['Event'][i]
  if '-' in event_name:
    event_name = event_name.replace('-', '_')
  if '.' in event_name:
    event_name = event_name.replace('.', '_')
  event_name = '_' + event_name
  globals()[event_name] = Event(DATA['Catalog'][i], DATA['Run'][i], DATA['Detector'][i], DATA['Event'][i], DATA['Time'][i], DATA['gps'][i])

In [50]:
print(datasets.find_datasets(type='events'))

['151008-v1', '151012.2-v1', '151116-v1', '161202-v1', '161217-v1', '170208-v1', '170219-v1', '170405-v1', '170412-v1', '170423-v1', '170616-v1', '170630-v1', '170705-v1', '170720-v1', '190924_232654-v1', '191118_212859-v1', '191223_014159-v1', '191225_215715-v1', '200114_020818-v1', '200121_031748-v1', '200201_203549-v1', '200214_224526-v1', '200214_224526-v2', '200219_201407-v1', '200311_103121-v1', 'GRB051103-v1', 'GW150914-v1', 'GW150914-v2', 'GW150914-v3', 'GW151012-v1', 'GW151012-v2', 'GW151012-v3', 'GW151226-v1', 'GW151226-v2', 'GW170104-v1', 'GW170104-v2', 'GW170608-v1', 'GW170608-v2', 'GW170608-v3', 'GW170729-v1', 'GW170809-v1', 'GW170814-v1', 'GW170814-v2', 'GW170814-v3', 'GW170817-v1', 'GW170817-v2', 'GW170817-v3', 'GW170818-v1', 'GW170823-v1', 'GW190403_051519-v1', 'GW190408_181802-v1', 'GW190408_181802-v2', 'GW190408_181802-v3', 'GW190412-v1', 'GW190412-v2', 'GW190412-v3', 'GW190412-v5', 'GW190412_053044-v4', 'GW190413_052954-v1', 'GW190413_052954-v2', 'GW190413_052954-v3'

In [56]:
print(type(_151012_2_v1.Detector))

<class 'set'>


In [52]:
DATA['Object_Name'] = '_' + DATA['Event']
for i in range(len(DATA)):
  DATA.loc[i, 'Object_Name'] = DATA['Object_Name'][i].replace('-', '_')
  DATA.loc[i, 'Object_Name'] = DATA['Object_Name'][i].replace('.', '_')
print(DATA['Object_Name'])

0      _blind_injection_v1
1             _GW150914_v1
2             _GW150914_v1
3             _GW150914_v2
4             _GW150914_v2
              ...         
704    _GW240107_013215_v1
705    _GW240109_050431_v1
706    _GW240109_050431_v1
707    _GW250114_082203_v1
708    _GW250114_082203_v1
Name: Object_Name, Length: 709, dtype: object


In [68]:

for i in range(len(DATA)):
  print(globals()[DATA['Object_Name'][i]].Event)
  #globals() is the dictionary
  #globals()['name'] is the variable in the dictionary with name 'name'
  #globals()['name'].Event is the Event attribute of the variable (object) 'name' in the case that the variable is an instance of the class Event (so an object)

blind_injection-v1
GW150914-v1
GW150914-v1
GW150914-v2
GW150914-v2
GW150914-v3
GW150914-v3
151008-v1
151008-v1
151012.2-v1
151012.2-v1
GW151012-v1
GW151012-v1
GW151012-v2
GW151012-v2
GW151012-v3
GW151012-v3
151116-v1
151116-v1
GW151226-v1
GW151226-v1
GW151226-v2
GW151226-v2
161202-v1
161202-v1
161217-v1
161217-v1
GW170104-v1
GW170104-v1
GW170104-v2
GW170104-v2
170208-v1
170208-v1
170219-v1
170219-v1
170405-v1
170405-v1
170412-v1
170412-v1
170423-v1
170423-v1
GW170608-v1
GW170608-v1
GW170608-v2
GW170608-v2
GW170608-v3
GW170608-v3
170616-v1
170616-v1
170630-v1
170630-v1
170705-v1
170705-v1
170720-v1
170720-v1
GW170729-v1
GW170729-v1
GW170809-v1
GW170809-v1
GW170814-v1
GW170814-v1
GW170814-v2
GW170814-v2
GW170814-v3
GW170814-v3
GW170817-v1
GW170817-v1
GW170817-v2
GW170817-v2
GW170817-v3
GW170817-v3
GW170818-v1
GW170818-v1
GW170823-v1
GW170823-v1
GW190403_051519-v1
GW190403_051519-v1
GW190408_181802-v2
GW190408_181802-v2
GW190408_181802-v1
GW190408_181802-v1
GW190408_181802-v3
GW190408_181